코랩에서 작업

In [26]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


FileExistsError: ignored

In [27]:
cd /content/mnt/My Drive/Colab Notebooks/pytorch-sentiment-analysis-kor-master

/content/mnt/My Drive/Colab Notebooks/pytorch-sentiment-analysis-kor-master


In [2]:
import numpy as np
import pandas as pd
import ast

In [28]:
cleaned_mbc_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_mbc_0816_0822.csv')

In [29]:
cleaned_mbc_0816_0822.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           407 non-null    int64  
 1   published_time       407 non-null    object 
 2   video_id             407 non-null    object 
 3   video_url            407 non-null    object 
 4   crawling_time        407 non-null    object 
 5   author               407 non-null    object 
 6   title                407 non-null    object 
 7   length               407 non-null    float64
 8   rating               407 non-null    float64
 9   views                407 non-null    float64
 10  thumbnail_url        407 non-null    object 
 11  captions             407 non-null    object 
 12  description          407 non-null    object 
 13  comments             407 non-null    object 
 14  article_url          407 non-null    object 
 15  cleaned_description  407 non-null    obj

In [30]:
a = ast.literal_eval(cleaned_mbc_0816_0822.comments[0])

In [31]:
a

["'카토오 기요마사'의 상을 세우려고 했다고?\n미쳤구먼.\n그 새끼 분명히 친일파 후손이다.",
 '친일파 척결이 필요하다',
 '왜장 동상은 에바지 \n미친거 아닌가??',
 '침략자 동상을 세운다고? 침략자 동상 세우는 자체도 문제이지만.\n동상이라는 것은 뭔가 배울 것 있고 훌륭한 사람을 기리기 위해 세우는 것이다.\n범죄자다. 범죄자 동상 세워서 범죄를 배우겠다는 것인가.\n참 한심하고 쓸모 없는 짓거리한다.',
 '얼마나 많은곳에 토착왜구들이 숨어있는거냐',
 '아니, 해방된지가 언젠데, 문체부가 생긴지가 언젠데, 이게 도대체 실홥니까?! 상식적으로 도저히 납득이 안되네요',
 '일본도 문화재급으로 취급 안하는데\n왜  우리가 보존하고 보수 하나?\n더군다나 우리 문화재는 경시 하면서 까지',
 '어느나라가 침략자 동상세워서 그들의 터를 문화재로 지정해 수십년 관리하는 \n미친놈들이 어디있어 . . .\n땅에서 하늘에서 조상들이 땅을 치고 울겠다~',
 '울산 중구청은 친일파였구나~~. 동상을 세우는거 반대하니까 이참에 뭐 일본식 신사도 지어놓지 그렇냐?',
 '현충원에도 친일파 많이 묻혀있다..\n파묘해야됨..',
 '울산시 실화냐? 동상세우려고 했다는게? 가토 기요마사가 학살한 우리 선조들 피는 태화강을 가득 채우고도 남는다. 주민 반대때문이 아니라 최소한 정상적인 사고를 가지고 행정을 해야지!!!',
 '정신나간기관이또있구나\n사법푸만문제인줄알았는데\n문화재청도그러나',
 '토착왜구를 박멸해야 하는 이유',
 '저런거에 아까운 국민세금 쓰이는구나 저 구청장은 정신이 온전한사람인지  의심스럽내',
 '그동안 우리나라 문화재에 관심이나 있었을까요 아직까지 일본식민지근성이 남아있는건지 의문스럽다',
 '중앙일보 사주 홍석현은 일본에 강제징용 배상금 요구하지 말라고 한다. 홍석현 외할배가 악질 친일매국노 김신석 (걱정말고 일제 정신대 댕겨오라고 연설).',
 '한국이야? 일본이야?\n한국은 맞는데 사람이 일본 사람인가?',
 '잘좀 하자. 우리 성 

In [32]:
a[0]

"'카토오 기요마사'의 상을 세우려고 했다고?\n미쳤구먼.\n그 새끼 분명히 친일파 후손이다."

# main

In [3]:
pip install soynlp

     |████████████████████████████████| 419kB 8.0MB/s 


In [17]:
pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 4.3MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [8]:
import argparse

from trainer import Trainer
from utils import load_dataset, make_iter, pad_sentence

In [12]:
import easydict

config_train = easydict.EasyDict({
    'batch_size':64,
    'num_epoch':7,
    'random_seed':32,

    # Model Hyper-parameters
    'lr':0.003,
    'vocab_size':25000,
    'embed_dim':100,
    'hidden_dim':256,
    'output_dim':1,
    'n_layer':2,
    'dropout':0.5,
    'bidirectional':True,

    # Options for CNN model
    'n_filters':100,
    'filter_sizes':[2, 3, 4],

    # Additional options
    'model':'vanilla_rnn',
    'optim':'Adam',
    'mode':'train',
    'save_model':'model.pt'
})

In [13]:
def main(config_train):
    if config_train.mode == 'train':
        train_data, valid_data = load_dataset(config_train.mode, config_train.random_seed)

        # if use CNN model, pad sentences to let all the batch inputs has minimum length (filter_sizes[-1])
        if config_train.model == 'cnn':
            train_data = pad_sentence(train_data, config_train.filter_sizes[-1])
            valid_data = pad_sentence(valid_data, config_train.filter_sizes[-1])

        train_iter, valid_iter, pad_idx = make_iter(config_train.batch_size, config_train.mode, train_data=train_data,
                                                    valid_data=valid_data)

        trainer = Trainer(config_train, pad_idx, train_iter=train_iter, valid_iter=valid_iter)

        trainer.train()

    else:
        test_data = load_dataset(config_train.mode, config_train.random_seed)
        if config_train.model == 'cnn':
            test_data = pad_sentence(test_data, config_train.filter_sizes[-1])
        test_iter, pad_idx = make_iter(config_train.batch_size, config_train.mode, test_data=test_data)
        trainer = Trainer(config_train, pad_idx, test_iter=test_iter)

        trainer.inference()

In [14]:
main(config_train)

Load NSMC dataset and convert it to pandas DataFrame . . .
Number of training examples: 104996
Number of validation examples: 44999
Make Iterators for training . . .
The model has 2,592,105 trainable parameters
RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)
Epoch: 01 | Epoch Time: 1m 1s
	Train Loss: 0.552 | Train Acc: 70.88%
	Val. Loss: 0.477 | Val. Acc: 77.08%
Epoch: 02 | Epoch Time: 1m 1s
	Train Loss: 0.419 | Train Acc: 80.70%
	Val. Loss: 0.425 | Val. Acc: 80.28%
Epoch: 03 | Epoch Time: 1m 2s
	Train Loss: 0.360 | Train Acc: 84.02%
	Val. Loss: 0.404 | Val. Acc: 81.67%
Epoch: 04 | Epoch Time: 1m 1s
	Train Loss: 0.327 | Train Acc: 85.73%
	Val. Loss: 0.414 | Val. Acc: 81.04%
Epoch: 05 | Epoch Time: 1m 0s
	Train Loss: 0.301 | Train Acc: 87.01%
	Val. Loss: 0.411 | Val. Acc: 81.25%
Epoch: 06 | Epoch Time: 1m 1s
	Train Loss: 0.281 | Train Acc: 88.20%
	Val. Loss: 0.412 | Val. Acc: 81.98%
Epoch: 07 | Epoch Time: 1m 

# predict

In [33]:
import pickle
import argparse
import torch

from soynlp.tokenizer import LTokenizer
from models.vanilla_rnn import RNN
from models.bidirectional_lstm import BidirectionalLSTM

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
config_predict = easydict.EasyDict({
    'lr':0.003,
    'vocab_size':25000,
    'embed_dim':100,
    'hidden_dim':256,
    'output_dim':1,
    'n_layer':2,
    'dropout':0.5,
    'bidirectional':True,

    'n_filters':100,
    'filter_sizes':[3, 4, 5],

    'model':'vanilla_rnn',
    'input':a[0],
    'save_model':'model.pt'
})

In [39]:
def predict_sequential(config_predict):
    # load tokenizer and torchtext Field
    pickle_tokenizer = open('pickles/tokenizer.pickle', 'rb')
    cohesion_scores = pickle.load(pickle_tokenizer)
    tokenizer = LTokenizer(scores=cohesion_scores)

    pickle_vocab = open('pickles/text.pickle', 'rb')
    text = pickle.load(pickle_vocab)
    pad_idx = text.vocab.stoi[text.pad_token]

    model_type = {
        'vanilla_rnn': RNN(config_predict, pad_idx),
        'bidirectional_lstm': BidirectionalLSTM(config_predict, pad_idx),
    }

    # select model and load trained model
    model = model_type[config_predict.model]
    model.load_state_dict(torch.load(config_predict.save_model))
    model.eval()

    # convert input into tensor and forward it through selected model
    tokenized = tokenizer.tokenize(config_predict.input)
    indexed = [text.vocab.stoi[token] for token in tokenized]
    length = [len(indexed)]

    tensor = torch.LongTensor(indexed).to(device)    # [input length]
    tensor = tensor.unsqueeze(1)                     # [input length, 1] for adding batch dimension
    length_tensor = torch.LongTensor(length)

    prediction = torch.sigmoid(model(tensor, length_tensor))
    label = torch.round(prediction)

    if label == 1:
        label = 'Positive'
    else:
        label = 'Negative'

    sentiment_percent = prediction.item()
    print(f'[in]  >> {config_predict.input}')
    print(f'[out] >> {sentiment_percent*100:.2f} % : {label}')

In [40]:
predict_sequential(config_predict)

[in]  >> '카토오 기요마사'의 상을 세우려고 했다고?
미쳤구먼.
그 새끼 분명히 친일파 후손이다.
[out] >> 13.03 % : Negative


In [42]:
def predict_sentiment(config_predict):
    # load tokenizer and torchtext Field
    pickle_tokenizer = open('pickles/tokenizer.pickle', 'rb')
    cohesion_scores = pickle.load(pickle_tokenizer)
    tokenizer = LTokenizer(scores=cohesion_scores)

    pickle_vocab = open('pickles/text.pickle', 'rb')
    text = pickle.load(pickle_vocab)
    pad_idx = text.vocab.stoi[text.pad_token]

    model_type = {
        'vanilla_rnn': RNN(config_predict, pad_idx),
        'bidirectional_lstm': BidirectionalLSTM(config_predict, pad_idx),
    }

    # select model and load trained model
    model = model_type[config_predict.model]
    model.load_state_dict(torch.load(config_predict.save_model))
    model.eval()

    # convert input into tensor and forward it through selected model
    tokenized = tokenizer.tokenize(config_predict.input)
    indexed = [text.vocab.stoi[token] for token in tokenized]
    length = [len(indexed)]

    tensor = torch.LongTensor(indexed).to(device)    # [input length]
    tensor = tensor.unsqueeze(1)                     # [input length, 1] for adding batch dimension
    length_tensor = torch.LongTensor(length)

    prediction = torch.sigmoid(model(tensor, length_tensor))
    # label = torch.round(prediction)

    sentiment_percent = prediction.item()
    
    return sentiment_percent

In [43]:
temp_value = predict_sentiment(config_predict)
temp_value

0.13029971718788147

In [44]:
cleaned_channel_a_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_channel_a_0816_0822.csv')
cleaned_jtbc_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_jtbc_0816_0822.csv')
cleaned_kbs_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_kbs_0816_0822.csv')
cleaned_mbc_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_mbc_0816_0822.csv')
cleaned_mbn_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_mbn_0816_0822.csv')
cleaned_sbs_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_sbs_0816_0822.csv')
cleaned_yonhab_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_yonhab_0816_0822.csv')
cleaned_ytn_0816_0822 = pd.read_csv('youtube_0816_0822/cleaned_data/cleaned_ytn_0816_0822.csv')

In [93]:
def add_comments_sentiment(DataFrame):
  '''DataFrame에서 comments들의 감성(긍정 또는 부정)을 분석하여 각각의 감성 및 평균적인 감성 (대중의 반응)을 산출
  
  DataFrame : cleaned_youtube_data
  '''
  copy = DataFrame.copy()
  
  # 저장하기 위한 리스트
  each_comments_sentiment_list = [] # 각 댓글들의 긍정 확률
  average_comments_sentiment_list = [] # 게시물의 평균 긍정 확률
  comments_sentiment_label_list = [] # 긍정 또는 부정 결과
  comments_sentiment_result_list = [] # 긍정 또는 부정 결과에 따른 출력 문구

  for element in copy.comments:
    comment_list = ast.literal_eval(element)
    temp_comments_sentiment_list = []

    for comment in comment_list:
      # 감성 예측을 위한 dict
      config_predict = easydict.EasyDict({
          'lr':0.003,
          'vocab_size':25000,
          'embed_dim':100,
          'hidden_dim':256,
          'output_dim':1,
          'n_layer':2,
          'dropout':0.5,
          'bidirectional':True,

          'n_filters':100,
          'filter_sizes':[3, 4, 5],

          'model':'vanilla_rnn',
          'input':comment,
          'save_model':'model.pt'})
      
      # 각각의 감성 예측 후 저장, 
      each_sentiment = predict_sentiment(config_predict)
      temp_comments_sentiment_list.append(each_sentiment)

    each_comments_sentiment_list.append(temp_comments_sentiment_list)
    
    try:
      # 표현 강도에 따른 간접 투표 방식
      average = sum(temp_comments_sentiment_list) / len(temp_comments_sentiment_list)
      if average >= 0.5:
        label = '긍정적'
        result = '긍정적인 댓글 {}%'.format(round(100*round(average, 3), 3))
      else:
        label = '부정적'
        result = '부정적인 댓글 {}%'.format(round(100 - 100*round(average, 3), 3))

      average_comments_sentiment_list.append(average)
      comments_sentiment_label_list.append(label)
      comments_sentiment_result_list.append(result)

    except: # 게시물의 댓글이 없으면 zero-division 에러 발생
      average_comments_sentiment_list.append(None)
      comments_sentiment_label_list.append(None)
      comments_sentiment_result_list.append(None)
      
  # DataFrame에 저장
  copy['each_comments_sentiment'] = each_comments_sentiment_list
  copy['average_comments_sentiment'] = average_comments_sentiment_list
  copy['comments_sentiment_label'] = comments_sentiment_label_list
  copy['comments_sentiment_result'] = comments_sentiment_result_list

  copy.dropna(axis=0, inplace=True)
  copy.reset_index(drop=True, inplace=True)

  return copy

In [94]:
channel_a = add_comments_sentiment(cleaned_channel_a_0816_0822)
channel_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  71 non-null     int64  
 1   published_time              71 non-null     object 
 2   video_id                    71 non-null     object 
 3   video_url                   71 non-null     object 
 4   crawling_time               71 non-null     object 
 5   author                      71 non-null     object 
 6   title                       71 non-null     object 
 7   length                      71 non-null     float64
 8   rating                      71 non-null     float64
 9   views                       71 non-null     float64
 10  thumbnail_url               71 non-null     object 
 11  captions                    71 non-null     object 
 12  description                 71 non-null     object 
 13  comments                    71 non-nu

In [96]:
channel_a.comments_sentiment_result[:20]

0     부정적인 댓글 57.6%
1     부정적인 댓글 62.0%
2     긍정적인 댓글 53.7%
3     부정적인 댓글 70.9%
4     부정적인 댓글 71.7%
5     긍정적인 댓글 67.8%
6     부정적인 댓글 64.4%
7     부정적인 댓글 67.7%
8     부정적인 댓글 60.5%
9     부정적인 댓글 52.1%
10    부정적인 댓글 73.2%
11    긍정적인 댓글 54.7%
12    부정적인 댓글 66.2%
13    부정적인 댓글 58.6%
14    긍정적인 댓글 56.2%
15    부정적인 댓글 59.7%
16    부정적인 댓글 71.2%
17    부정적인 댓글 57.7%
18    긍정적인 댓글 51.7%
19    부정적인 댓글 63.2%
Name: comments_sentiment_result, dtype: object

In [97]:
channel_a.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_channel_a_0816_0822.csv')

In [98]:
jtbc = add_comments_sentiment(cleaned_jtbc_0816_0822)
jtbc.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_jtbc_0816_0822.csv')

In [99]:
kbs = add_comments_sentiment(cleaned_kbs_0816_0822)
kbs.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_kbs_0816_0822.csv')

In [100]:
mbc = add_comments_sentiment(cleaned_mbc_0816_0822)
mbc.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_mbc_0816_0822.csv')

In [101]:
mbn = add_comments_sentiment(cleaned_mbn_0816_0822)
mbn.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_mbn_0816_0822.csv')

In [102]:
sbs = add_comments_sentiment(cleaned_sbs_0816_0822)
sbs.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_sbs_0816_0822.csv')

In [103]:
yonhab = add_comments_sentiment(cleaned_yonhab_0816_0822)
yonhab.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_yonhab_0816_0822.csv')

In [104]:
ytn = add_comments_sentiment(cleaned_ytn_0816_0822)
ytn.to_csv('youtube_0816_0822/add_comment_sentiment_data/cs_ytn_0816_0822.csv')

In [105]:
ytn.comments_sentiment_result

0      부정적인 댓글 59.3%
1      부정적인 댓글 60.4%
2      부정적인 댓글 63.0%
3      부정적인 댓글 57.9%
4      부정적인 댓글 64.3%
           ...      
719    부정적인 댓글 64.2%
720    부정적인 댓글 59.0%
721    부정적인 댓글 51.4%
722    긍정적인 댓글 61.9%
723    부정적인 댓글 54.4%
Name: comments_sentiment_result, Length: 724, dtype: object